In [2]:
from nipype.interfaces import fsl
from nipype.interfaces.fsl import MultipleRegressDesign
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe  
from nipype import Workflow, Node, MapNode, IdentityInterface, Function, DataSink, JoinNode, SelectFiles
import os
import os.path as op
import argparse
import glob
from bids import BIDSLayout
import pandas as pd

In [2]:
proj_dir = '/nese/mit/group/saxelab/projects/EMOfd/'
master_data = op.join(proj_dir, 'data/subject_lists/EMOfd_subject_info_211026.csv')

df = pd.read_csv(master_data)

participant_IDs = pd.unique(df.loc[(df.exclude_from_analysis == False), 'participantID']).tolist()
for ID in participant_IDs:
    df.loc[df['participantID'] == ID, 'exclude_whole_row'] = False
value_counts = df[(df['exclude_whole_row'] == False) & (df['match'] == 1.0)].group.value_counts()
endorse_count = value_counts['ENDORSE']
oppose_count = value_counts['OPPOSE']

In [3]:
### Change this for your study

# From the docs: (https://nipype.readthedocs.io/en/v0.14.0/interfaces/generated/interfaces.fsl/model.html)
# Contrasts: List of contrasts with each contrast being a list of the form -
        # [('name', 'stat', [condition list], [weight list])]
# Groups: dictionary containing named lists of regressors
  
    
    
## !! IMPORTANT !! 

# BASED ON DEFAULT GROUP ORDER IN whole_brain.ipynb: 

# for endorse > oppose, contrast weights should be [1, -1]
# for oppose > endorse, contrast weights should be [-1, 1]

contrasts = [
        ['endorse_gt_oppose', 'T', ['ENDORSE', 'OPPOSE'], [1, -1]]
    ]

# (otherwise can switch and run ...)

#contrasts = [
#        ['oppose_gt_endorse', 'T', ['ENDORSE', 'OPPOSE'], [-1, 1]]
#    ]


    
    
groups = dict(
        ENDORSE = [1] * endorse_count + [0] * oppose_count,
        OPPOSE = [0] * endorse_count + [1] * oppose_count,   
    )


flame_dir = proj_dir+ '/TIER/analysis_data/group_flame'

tasks = ['read']

outdir = 'group_analysis_output_oppose_gt_endorse_randomise'

###
"oppose: {} + endorse: {} = {}".format(oppose_count, endorse_count, oppose_count + endorse_count)

'oppose: 21 + endorse: 21 = 42'

In [ ]:
def create_secondlevel_workflow(data_dir, flame_dir, task, outdir, mask, name="task-{}_workingdir"):

    con, between_or_all = data_dir.split(task)[1][1:].rsplit('_', 1)
    print(between_or_all)
    
    # Initialize workflow
    wf = pe.Workflow(name=name.format(task),
                    base_dir=flame_dir)
    
    # group, con, mat
    l2model = pe.Node(interface=MultipleRegressDesign(),
                      name='l2model')
    
    l2model.inputs.contrasts = contrasts
    l2model.inputs.regressors = groups
    
    # copes and varcopes
        
    templates2 = {'all_copes': 'all_copes*',
                     'all_varcopes': 'all_varcopes*'}
    get_runs = Node(SelectFiles(templates2),
                name='selectruns')
    get_runs.inputs.base_directory = op.join(flame_dir, data_dir)
    
    # maskfile
    templates = {'maskfile': mask}
    get_mask = Node(SelectFiles(templates),
                  name='selectmask')
    get_mask.inputs.base_directory = ''
    
    datasink = pe.Node(nio.DataSink(), name='sinker')
    datasink.inputs.base_directory=op.join(flame_dir, outdir)
    new_dir = "randomise_{}_{}".format(task, con)
    datasink.inputs.regexp_substitutions = [
        ("stats_dir", ""), 
        ("stats", new_dir),
        ('cov_split_file', new_dir),
        ('mask_file', new_dir),
        ('var_cope_file', new_dir),
        ('cope_file', new_dir),
        ('design_file', new_dir),
        ('t_con_file', new_dir)
    ]

    wf.connect([ 
        (get_runs,datasink,[('all_copes','cope_file'), 
                                  ('all_varcopes', 'var_cope_file')]),
        (l2model,datasink, [('design_mat','design_file'),
                                ('design_con','t_con_file'),
                                ('design_grp','cov_split_file')]),
        (get_mask,datasink,[('maskfile','mask_file')])
               ])
    
    return wf

In [ ]:
mask = '/cm/shared/openmind/fsl/5.0.9/data/standard/avg152T1_brain.nii.gz'

for task in tasks:
    for data_dir in glob.glob('{}/{}*allsubs*'.format(flame_dir, task)):
        wf = create_secondlevel_workflow(data_dir, flame_dir, task, outdir, mask)
        wf.config['execution'] = {'crashfile_format': 'txt',
                                    'remove_unnecessary_outputs': False,
                                    'keep_inputs': True}

        results = wf.run(plugin='MultiProc')
        print("Task '{}' is finished".format(task))
        